In [1]:
from notebook.graph import ModelDisplay

%matplotlib inline
from model.projects import StandardProject
from IPython.core.display_functions import display

p = StandardProject()
g = ModelDisplay(p)
display(g.inputs.get_widget(), g.output.get_widget())

Output()

In [2]:
import ipywidgets as widgets

a = widgets.IntSlider()
b = widgets.IntSlider()
c = widgets.IntSlider()
ui = widgets.HBox([a, b, c])


def f(a, b, c):
    print((a, b, c))


out = widgets.interactive_output(f, {"a": a, "b": b, "c": c})

display(ui, out)

Output()